## **TRANSFORMACIÓN A MODELO DE CLASIFICACIÓN**

GRUPO DE PRÁCTICAS: 82
- AITANA ANTONIA ORTIZ GUIÑO    100472097
- MARÍA PARRA MUÑOZ             100472195
ENLACE AL REPOSITORIO: https://github.com/aitanax/practica

In [281]:
import pandas as pd
import numpy as np
import numpy as df
import seaborn as sns
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.model_selection import TimeSeriesSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, mean_squared_error, mean_absolute_error, r2_score, confusion_matrix, balanced_accuracy_score

import warnings
warnings.filterwarnings('ignore')


In [282]:
wind_ava = pd.read_csv('wind_ava.csv.gz', compression="gzip", sep=",")

In [283]:
sotavento_columns = ['datetime', 'energy']
sotavento_columns.extend(col for col in wind_ava.columns if col.endswith(".13"))
wind_ava_sotavento = wind_ava[sotavento_columns]
wind_ava_sotavento.head(10)

,datetime,energy,p54.162.13,p55.162.13,cape.13,p59.162.13,lai_lv.13,lai_hv.13,u10n.13,v10n.13,...,t2m.13,stl2.13,stl3.13,iews.13,inss.13,stl4.13,fsr.13,flsr.13,u100.13,v100.13
0,2005-01-02 18:00:00,402.71,2.510824e+06,9.186295,13.527577,1.386937e+06,2.344111,2.432983,-0.757587,-1.922799,...,280.473098,281.042026,281.462478,-0.057958,-0.138650,284.684755,0.404731,-5.927092,-1.780562,-4.443617
1,2005-01-03 00:00:00,696.80,2.513173e+06,8.849569,6.896412,1.153526e+06,2.343719,2.432838,-1.412620,-1.403011,...,278.286616,280.747406,281.486541,-0.103576,-0.083050,284.667948,0.404920,-5.913881,-3.743344,-3.129469
2,2005-01-03 06:00:00,1591.15,2.509627e+06,7.924080,4.774439,1.098754e+06,2.343300,2.432704,-2.290185,-0.754580,...,277.206490,280.114863,281.487095,-0.165721,-0.036241,284.651914,0.405704,-5.908272,-5.097203,-1.157748
3,2005-01-03 12:00:00,1338.62,2.510571e+06,6.922709,0.000000,1.076021e+06,2.342830,2.432514,-3.497855,1.271028,...,280.926600,279.991138,281.472435,-0.275550,0.098192,284.636266,0.403967,-5.961995,-4.500835,1.502478
4,2005-01-03 18:00:00,562.50,2.505664e+06,6.646282,0.000000,1.070830e+06,2.342437,2.432369,-0.971249,0.553060,...,277.363875,280.576898,281.473265,-0.056553,0.041844,284.620232,0.403808,-5.987860,-3.392324,2.131114
5,2005-01-04 00:00:00,232.30,2.505768e+06,6.125948,0.000000,1.094100e+06,2.342045,2.432235,-0.153568,1.554043,...,274.850965,279.654761,281.464691,0.003361,0.144389,284.603425,0.404758,-5.957546,0.190590,5.349629
6,2005-01-04 06:00:00,329.95,2.503477e+06,7.038564,0.000000,1.264504e+06,2.341653,2.432090,1.247802,1.676775,...,275.490996,278.712132,281.416011,0.115504,0.144389,284.585652,0.406001,-5.957198,4.105168,4.612229
7,2005-01-04 12:00:00,960.51,2.504995e+06,7.938308,0.000000,1.247857e+06,2.341208,2.431944,2.582634,2.243500,...,281.785221,278.574102,281.349352,0.177567,0.201828,284.567493,0.405072,-5.981510,3.387434,3.809970
8,2005-01-04 18:00:00,194.62,2.503739e+06,8.447123,0.000000,1.525480e+06,2.340816,2.431810,1.169065,2.014377,...,280.623488,279.538769,281.309800,0.085340,0.126537,284.550879,0.405413,-5.995207,3.280716,4.545374
9,2005-01-05 00:00:00,358.51,2.505664e+06,8.417990,0.000000,1.620168e+06,2.340397,2.431665,0.808280,1.977174,...,277.964268,279.519437,281.288502,0.063358,0.111138,284.535038,0.407139,-5.965171,3.136332,4.321859


In [284]:
wind_ava_sotavento = wind_ava_sotavento.rename(columns = {
    "datetime": "Datetime",
    "energy": "Energy",
    "p54.162.13": "Vertical integral of temperature", 
    "p55.162.13": "Vertical integral of water vapour", 
    "cape.13": "Convective available potential energy", 
    "p59.162.13": "Vertical integral of divergence of kinetic energy", 
    "lai_lv.13": "Leaf area index, low vegetation", 
    "lai_hv.13": "Leaf area index, high vegetation", 
    "u10n.13": "Neutral wind at 10 m u-component", 
    "v10n.13": "Neutral wind at 10 m v-component", 
    "sp.13": "Surface pressure",
    "stl1.13": "Soil temperature level 1",
    "u10.13": "10 metre U wind component",
    "v10.13": "10 metre V wind component",
    "t2m.13": "2 metre temperature", 
    "stl2.13": "Soil temperature level 2", 
    "stl3.13": "Soil temperature level 3", 
    "iews.13": "Instantaneous eastward turbulent surface stress",
    "inss.13": "Instantaneous northward turbulent surface", 
    "stl4.13": "Soil temperature level 4", 
    "fsr.13": "Forecast surface roughness", 
    "flsr.13": "Forecast logarithm of surface roughness for heat",
    "u100.13": "100 metre U wind component",
    "v100.13": "100 metre V wind component", })
wind_ava_sotavento.head()

,Datetime,Energy,Vertical integral of temperature,Vertical integral of water vapour,Convective available potential energy,Vertical integral of divergence of kinetic energy,"Leaf area index, low vegetation","Leaf area index, high vegetation",Neutral wind at 10 m u-component,Neutral wind at 10 m v-component,...,2 metre temperature,Soil temperature level 2,Soil temperature level 3,Instantaneous eastward turbulent surface stress,Instantaneous northward turbulent surface,Soil temperature level 4,Forecast surface roughness,Forecast logarithm of surface roughness for heat,100 metre U wind component,100 metre V wind component
0,2005-01-02 18:00:00,402.71,2.510824e+06,9.186295,13.527577,1.386937e+06,2.344111,2.432983,-0.757587,-1.922799,...,280.473098,281.042026,281.462478,-0.057958,-0.138650,284.684755,0.404731,-5.927092,-1.780562,-4.443617
1,2005-01-03 00:00:00,696.80,2.513173e+06,8.849569,6.896412,1.153526e+06,2.343719,2.432838,-1.412620,-1.403011,...,278.286616,280.747406,281.486541,-0.103576,-0.083050,284.667948,0.404920,-5.913881,-3.743344,-3.129469
2,2005-01-03 06:00:00,1591.15,2.509627e+06,7.924080,4.774439,1.098754e+06,2.343300,2.432704,-2.290185,-0.754580,...,277.206490,280.114863,281.487095,-0.165721,-0.036241,284.651914,0.405704,-5.908272,-5.097203,-1.157748
3,2005-01-03 12:00:00,1338.62,2.510571e+06,6.922709,0.000000,1.076021e+06,2.342830,2.432514,-3.497855,1.271028,...,280.926600,279.991138,281.472435,-0.275550,0.098192,284.636266,0.403967,-5.961995,-4.500835,1.502478
4,2005-01-03 18:00:00,562.50,2.505664e+06,6.646282,0.000000,1.070830e+06,2.342437,2.432369,-0.971249,0.553060,...,277.363875,280.576898,281.473265,-0.056553,0.041844,284.620232,0.403808,-5.987860,-3.392324,2.131114


In [285]:
def classificate_energy(value):
    if value <= tercer_cuartil:
        return 0
    else:
        return 1
tercer_cuartil = wind_ava_sotavento["Energy"].quantile(0.75)
wind_ava_sotavento["Energy_Class"] = wind_ava_sotavento["Energy"].apply(classificate_energy)
print("El tercer cuartil es:", tercer_cuartil)  

wind_ava_sotavento.head(10)

El tercer cuartil es: 1089.375


,Datetime,Energy,Vertical integral of temperature,Vertical integral of water vapour,Convective available potential energy,Vertical integral of divergence of kinetic energy,"Leaf area index, low vegetation","Leaf area index, high vegetation",Neutral wind at 10 m u-component,Neutral wind at 10 m v-component,...,Soil temperature level 2,Soil temperature level 3,Instantaneous eastward turbulent surface stress,Instantaneous northward turbulent surface,Soil temperature level 4,Forecast surface roughness,Forecast logarithm of surface roughness for heat,100 metre U wind component,100 metre V wind component,Energy_Class
0,2005-01-02 18:00:00,402.71,2.510824e+06,9.186295,13.527577,1.386937e+06,2.344111,2.432983,-0.757587,-1.922799,...,281.042026,281.462478,-0.057958,-0.138650,284.684755,0.404731,-5.927092,-1.780562,-4.443617,0
1,2005-01-03 00:00:00,696.80,2.513173e+06,8.849569,6.896412,1.153526e+06,2.343719,2.432838,-1.412620,-1.403011,...,280.747406,281.486541,-0.103576,-0.083050,284.667948,0.404920,-5.913881,-3.743344,-3.129469,0
2,2005-01-03 06:00:00,1591.15,2.509627e+06,7.924080,4.774439,1.098754e+06,2.343300,2.432704,-2.290185,-0.754580,...,280.114863,281.487095,-0.165721,-0.036241,284.651914,0.405704,-5.908272,-5.097203,-1.157748,1
3,2005-01-03 12:00:00,1338.62,2.510571e+06,6.922709,0.000000,1.076021e+06,2.342830,2.432514,-3.497855,1.271028,...,279.991138,281.472435,-0.275550,0.098192,284.636266,0.403967,-5.961995,-4.500835,1.502478,1
4,2005-01-03 18:00:00,562.50,2.505664e+06,6.646282,0.000000,1.070830e+06,2.342437,2.432369,-0.971249,0.553060,...,280.576898,281.473265,-0.056553,0.041844,284.620232,0.403808,-5.987860,-3.392324,2.131114,0
5,2005-01-04 00:00:00,232.30,2.505768e+06,6.125948,0.000000,1.094100e+06,2.342045,2.432235,-0.153568,1.554043,...,279.654761,281.464691,0.003361,0.144389,284.603425,0.404758,-5.957546,0.190590,5.349629,0
6,2005-01-04 06:00:00,329.95,2.503477e+06,7.038564,0.000000,1.264504e+06,2.341653,2.432090,1.247802,1.676775,...,278.712132,281.416011,0.115504,0.144389,284.585652,0.406001,-5.957198,4.105168,4.612229,0
7,2005-01-04 12:00:00,960.51,2.504995e+06,7.938308,0.000000,1.247857e+06,2.341208,2.431944,2.582634,2.243500,...,278.574102,281.349352,0.177567,0.201828,284.567493,0.405072,-5.981510,3.387434,3.809970,0
8,2005-01-04 18:00:00,194.62,2.503739e+06,8.447123,0.000000,1.525480e+06,2.340816,2.431810,1.169065,2.014377,...,279.538769,281.309800,0.085340,0.126537,284.550879,0.405413,-5.995207,3.280716,4.545374,0
9,2005-01-05 00:00:00,358.51,2.505664e+06,8.417990,0.000000,1.620168e+06,2.340397,2.431665,0.808280,1.977174,...,279.519437,281.288502,0.063358,0.111138,284.535038,0.407139,-5.965171,3.136332,4.321859,0


Cambiamos la forma de partir los datos, a una más sencilla con respecto al modelo de clasficación. Extrae los datos con la misma precisión.

In [286]:
X = wind_ava_sotavento.iloc[:, 2:24]
Y = wind_ava_sotavento.iloc[:, 24]

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=1099, random_state=100472097, shuffle=False)

# **MÉTRICAS SELECCIONADAS:**

Las métricas seleccionadas  para evaluar y comparar los modelos son una medida cuantitativa del rendimiento del modelo en la tarea de clasificación. Aquí hay una explicación de por qué elegimos cada una de ellas:

1. **Accuracy (Precisión)**:
   - Es una medida general de la capacidad del modelo para clasificar correctamente las instancias en todas las clases.
   - Es fácil de interpretar y entender: simplemente indica la proporción de predicciones correctas sobre el total de predicciones.

2. **Recall (Sensibilidad o True Positive Rate)**:
   - Mide la proporción de instancias positivas reales que fueron correctamente identificadas por el modelo.
   - Es importante en situaciones donde la identificación de instancias positivas es crítica y queremos minimizar los falsos negativos.

3. **True Negative Rate (Especificidad)**:
   - Mide la proporción de instancias negativas reales que fueron correctamente identificadas por el modelo.
   - Es importante en situaciones donde la identificación de instancias negativas es crítica y queremos minimizar los falsos positivos.

4. **Balanced Accuracy (Precisión equilibrada)**:
   - Es la media de la sensibilidad y la especificidad, proporcionando una visión equilibrada del rendimiento del modelo en todas las clases.
   - Es útil cuando las clases están desequilibradas y queremos asegurarnos de que el modelo funcione bien en todas las clases.

5. **F1 Score**:
   - Es una medida que combina precisión y recall en una sola métrica, proporcionando una evaluación general del rendimiento del modelo.
   - Es útil cuando hay un desequilibrio entre las clases o cuando queremos un balance entre precisión y recall.

En resumen, estas métricas nos dan una comprensión completa del rendimiento del modelo en términos de su capacidad para clasificar correctamente las instancias positivas y negativas, así como su equilibrio general entre precisión y recall. Utilizamos estas métricas para comparar y determinar cuál modelo tiene el mejor rendimiento en la tarea de clasificación específica que estamos abordando.

# **MEJOR MODELO A MODELO DE CLASIFICACIÓN**

Lo primero que hacemos en esta sección es con el mejor modelo que obtuvimos en regresión lo pasamos a clasificación

## SOBRE EL BALANCEADO:

En el contexto de la clasificación binaria, esto significa que el número de ejemplos en cada clase (positiva y negativa) es aproximadamente el mismo o está ajustado de manera que el modelo no esté sesgado hacia una clase dominante.

Cuando no se balancea el modelo, el conjunto de datos puede tener una distribución desigual de las clases, lo que puede llevar a que el modelo aprenda de manera sesgada, dando más peso o importancia a la clase dominante y menos a la clase minoritaria. Esto puede resultar en un modelo que tiene un buen rendimiento en términos de métricas globales (como la precisión) pero puede tener un rendimiento deficiente en la detección de la clase minoritaria.

Cuando se balancea el modelo, se pueden aplicar diferentes técnicas, como:

1. **Submuestreo de la clase dominante**: Reducir aleatoriamente el número de ejemplos de la clase dominante para que esté más equilibrado con la clase minoritaria.

2. **Sobremuestreo de la clase minoritaria**: Aumentar aleatoriamente el número de ejemplos de la clase minoritaria para equilibrar las clases.

3. **Generación sintética de muestras**: Crear nuevos ejemplos sintéticos para la clase minoritaria utilizando técnicas como SMOTE (Synthetic Minority Over-sampling Technique).

4. **Peso de clase**: Dar pesos diferentes a las clases durante el entrenamiento del modelo para compensar el desequilibrio de clases.

Por lo que lo que nosotras intendimos es que la diferencia principal entre balancear y no balancear el modelo es que, al balancearlo, se intenta reducir el impacto del desequilibrio de clases en el rendimiento del modelo, especialmente en la capacidad de generalización para predecir correctamente las clases minoritarias. Por otro lado, si no se balancea el modelo, existe el riesgo de que el modelo esté sesgado hacia la clase dominante y tenga un rendimiento deficiente en la clasificación de la clase minoritaria.

## SVC con ajuste de hiperparámetros sin balancear

En este apartado se buscan los mejores hiperparámetros para un clasificador SVM utilizando una validación cruzada de 5-fold y luego evalúa el rendimiento del modelo con las métricas seleccionadas en un conjunto de datos de prueba. Para este primer modelo el clasificacion no se balancea, pero si se ajustan hiperparametros.

In [287]:
scaler = StandardScaler()
svc = SVC(random_state = 100472097)
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

clf_hpo = Pipeline(steps=[('scaler', scaler), ('svc', svc)])

slf_hpo_params = {'svc__C': [0.1, 1, 10], 'svc__gamma': [1, 0.1, 0.01, 10, 100], 'svc__kernel': ['linear','rbf']}

clf_hpo = GridSearchCV(clf_hpo, slf_hpo_params, cv=5, scoring='balanced_accuracy', n_jobs=-1)

clf_hpo.fit(x_train_scaled, y_train)
y_pred = clf_hpo.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accurancy = balanced_accuracy_score(y_test, y_pred)
f1_1 = f1_score(y_test, y_pred, average='weighted')

print('Los mejores hiperparametros son:', clf_hpo.best_params_)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accurancy: ", balanced_accurancy)
print("F1 Score: ", f1_1)

Los mejores hiperparametros son: {'svc__C': 10, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
Accuracy:  0.8717015468607825
Recall:  0.6549295774647887
True Negative Rate:  0.947239263803681
True Positive Rate:  0.6549295774647887
Balanced Accurancy:  0.8010844206342349
F1 Score:  0.8669178411387228


## SVC sin ajuste de hiperparámetros y balanceado

En este apartado no se incluyem consideraciones específicas de balanceamiento de clases. Y además no se ajustan hiperaparametros.

In [288]:
from sklearn.svm import SVC
scaler = StandardScaler()
svc = SVC(random_state = 100472097)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

clf_hpo = Pipeline(steps=[('scaler', scaler), ('svc', svc)])

clf_hpo.fit(x_train_scaled, y_train)
y_pred = clf_hpo.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=1)
tpr = recall_score(y_test, y_pred, pos_label=0)
balanced_accurancy = balanced_accuracy_score(y_test, y_pred)
f1_1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accurancy: ", balanced_accurancy)
print("F1 Score: ", f1_1)

Accuracy:  0.8671519563239308
Recall:  0.6619718309859155
True Negative Rate:  0.6619718309859155
True Positive Rate:  0.9386503067484663
Balanced Accurancy:  0.8003110688671908
F1 Score:  0.8631215921389412


## SVC con ajuste de hiperparámetros y balanceado

Aquí se implementa un proceso de ajuste de hiperparámetros para un modelo de clasificación utilizando SVM (Support Vector Machine) con consideraciones de balanceamiento de clases y validación cruzada.

In [289]:
scaler = StandardScaler()
svc = SVC(class_weight='balanced', random_state = 100472097)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

clf_hpo = Pipeline(steps=[('scaler', scaler), ('svc', svc)])

slf_hpo_params = {'svc__C': [0.1, 1, 10], 'svc__gamma': [1, 0.1, 0.01, 10, 100], 'svc__kernel': ['linear','rbf']}

clf_hpo = GridSearchCV(clf_hpo, slf_hpo_params, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)

clf_hpo.fit(x_train_scaled, y_train)
y_pred = clf_hpo.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accurancy = balanced_accuracy_score(y_test, y_pred)
f1_1 = f1_score(y_test, y_pred, average='weighted')

print('Los mejores hiperparametros son:', clf_hpo.best_params_)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accurancy: ", balanced_accurancy)
print("F1 Score: ", f1_1)

Los mejores hiperparametros son: {'svc__C': 1, 'svc__gamma': 0.01, 'svc__kernel': 'rbf'}
Accuracy:  0.8471337579617835
Recall:  0.8133802816901409
True Negative Rate:  0.8588957055214724
True Positive Rate:  0.8133802816901409
Balanced Accurancy:  0.8361379936058067
F1 Score:  0.8516335196498982


# ***Matriz de confusión en SVM***

La matriz de confusión es una herramienta útil para comprender el rendimiento del modelo en la clasificación de las clases.

In [290]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index = ["Real 0", "Real 1"], columns = [" Class Pred 0", "Class Pred 1"])

print(cm_df)

         Class Pred 0  Class Pred 1
Real 0            700           115
Real 1             53           231


Interpretación de la matriz de confusión:

Para la clase verdadera "0" (Real 0), el modelo predijo correctamente 700 instancias como "0" y erróneamente clasificó 115 instancias como "1".
Para la clase verdadera "1" (Real 1), el modelo predijo correctamente 231 instancias como "1" y erróneamente clasificó 53 instancias como "0".

# **MODELO DE CLASIFICACIÓN KNN**

Observaciones tras la obtención de resultados: 

Estos resultados comparan el rendimiento de un modelo de clasificación KNN (K-Nearest Neighbors) antes y después de ajustar sus hiperparámetros.

**Resultados del modelo sin ajuste de hiperparámetros:**
- Accuracy: 0.844
- Recall: 0.595
- True Negative Rate: 0.931
- True Positive Rate: 0.595
- Balanced Accuracy: 0.763
- F1 Score: 0.664

**Resultados del modelo con hiperparámetros ajustados:**
- Mejores hiperparámetros encontrados: {'knn__algorithm': 'auto', 'knn__n_neighbors': 13, 'knn__p': 1, 'knn__weights': 'distance'}
- Accuracy: 0.863
- Recall: 0.599
- True Negative Rate: 0.955
- True Positive Rate: 0.599
- Balanced Accuracy: 0.777
- F1 Score: 0.855

Comparando los dos conjuntos de resultados:

1. **Accuracy:** El modelo con hiperparámetros ajustados tiene una precisión ligeramente mayor (86.3% frente a 84.4%).
2. **Recall y True Positive Rate:** Ambos modelos tienen un rendimiento similar en términos de recall y True Positive Rate, con una ligera mejora en el modelo con hiperparámetros ajustados.
3. **True Negative Rate:** El modelo con hiperparámetros ajustados tiene una tasa de verdaderos negativos ligeramente mayor (95.5% frente a 93.1%).
4. **Balanced Accuracy:** La precisión equilibrada también es ligeramente mayor en el modelo con hiperparámetros ajustados (77.7% frente a 76.3%).
5. **F1 Score:** El modelo con hiperparámetros ajustados muestra una mejora significativa en la puntuación F1 (85.5% frente a 66.4%), lo que indica un mejor equilibrio entre precisión y recall.

En resumen, ajustar los hiperparámetros del modelo KNN ha mejorado su rendimiento general, especialmente en términos de F1 Score, lo que sugiere que es una configuración más óptima para este conjunto de datos y problema de clasificación.

In [291]:
scaler = StandardScaler()
knn_classifier = KNeighborsClassifier(n_neighbors=5)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

knn_classifier.fit(x_train_scaled, y_train)
y_pred = knn_classifier.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1_knn = f1_score(y_test, y_pred)

print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accuracy: ", balanced_accuracy)
print("F1 Score: ", f1_knn)


Accuracy:  0.8444040036396724
Recall:  0.5950704225352113
True Negative Rate:  0.9312883435582822
True Positive Rate:  0.5950704225352113
Balanced Accuracy:  0.7631793830467467
F1 Score:  0.6640471512770137


In [292]:
scaler = StandardScaler()
knn_classifier = KNeighborsClassifier()

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

pipe = Pipeline(steps=[('scaler', scaler), ('knn', knn_classifier)])

param_grid = {
    'knn__n_neighbors': [3, 5, 7, 13, 15, 30],
    'knn__weights': ['uniform', 'distance'],
    'knn__p': [1, 2], 
    'knn__algorithm': ['auto', 'ball_tree', 'brute', 'kd_tree']
}

knn_classifier = GridSearchCV(pipe, param_grid, cv=TimeSeriesSplit(n_splits=5))
knn_classifier.fit(x_train_scaled, y_train)
y_pred = knn_classifier.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1_knn = f1_score(y_test, y_pred, average='weighted')

print("Mejores hiperparámetros encontrados:", knn_classifier.best_params_)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accuracy: ", balanced_accuracy)
print("F1 Score: ", f1_knn)


Mejores hiperparámetros encontrados: {'knn__algorithm': 'auto', 'knn__n_neighbors': 13, 'knn__p': 1, 'knn__weights': 'distance'}
Accuracy:  0.8626023657870792
Recall:  0.5985915492957746
True Negative Rate:  0.9546012269938651
True Positive Rate:  0.5985915492957746
Balanced Accuracy:  0.7765963881448199
F1 Score:  0.8549277163642376


# ***Matriz de confusión en KNN***

La matriz de confusión es una herramienta útil para comprender el rendimiento del modelo en la clasificación de las clases.

In [293]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index = ["Real 0", "Real 1"], columns = [" Class Pred 0", "Class Pred 1"])

print(cm_df)

         Class Pred 0  Class Pred 1
Real 0            778            37
Real 1            114           170


En esta matriz de confusión:

Para la clase verdadera "0" (Real 0), el modelo predijo correctamente 778 instancias como "0" y erróneamente clasificó 37 instancias como "1".
Para la clase verdadera "1" (Real 1), el modelo predijo correctamente 170 instancias como "1" y erróneamente clasificó 114 instancias como "0".

# **MODELO DE CLASIFICACIÓN ÁRBOL DE CLASIFICACIÓN**

Observaciones tras la obtención de resultados:

Estos resultados comparan el rendimiento de un modelo de árbol de clasificación antes y después de ajustar sus hiperparámetros.

**Resultados del modelo sin ajuste de hiperparámetros:**
- Accuracy: 0.813
- Recall: 0.609
- True Negative Rate: 0.883
- True Positive Rate: 0.609
- Balanced Accuracy: 0.746
- F1 Score: 0.811

**Resultados del modelo con hiperparámetros ajustados:**
- Mejores hiperparámetros encontrados: {'tree__max_depth': 3, 'tree__min_samples_leaf': 2, 'tree__min_samples_split': 2}
- Accuracy: 0.846
- Recall: 0.680
- True Negative Rate: 0.904
- True Positive Rate: 0.680
- Balanced Accuracy: 0.792
- F1 Score: 0.845

Comparando los dos conjuntos de resultados:

1. **Accuracy:** El modelo con hiperparámetros ajustados tiene una precisión ligeramente mayor (84.6% frente a 81.3%).
2. **Recall y True Positive Rate:** El modelo con hiperparámetros ajustados tiene un recall y True Positive Rate más altos (68.0% frente a 60.9%), lo que indica una mejor capacidad para identificar la clase positiva.
3. **True Negative Rate:** El modelo con hiperparámetros ajustados también tiene una tasa de verdaderos negativos ligeramente mayor (90.4% frente a 88.3%).
4. **Balanced Accuracy:** La precisión equilibrada también es ligeramente mayor en el modelo con hiperparámetros ajustados (79.2% frente a 74.6%).
5. **F1 Score:** El modelo con hiperparámetros ajustados muestra una mejora en la puntuación F1 (84.5% frente a 81.1%), lo que indica un mejor equilibrio entre precisión y recall.

En resumen, ajustar los hiperparámetros del modelo de árbol de clasificación ha mejorado su rendimiento general, especialmente en términos de recall, Balanced Accuracy y F1 Score, lo que sugiere que es una configuración más óptima para este conjunto de datos y problema de clasificación.

In [294]:
scaler = StandardScaler()
tree= DecisionTreeClassifier(random_state = 100472097)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)


tree.fit(x_train_scaled, y_train)
y_pred = tree.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accurancy = balanced_accuracy_score(y_test, y_pred)
f1_1 = f1_score(y_test, y_pred, average='weighted')

print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accurancy: ", balanced_accurancy)
print("F1 Score: ", f1_1)

Accuracy:  0.8125568698817106
Recall:  0.6091549295774648
True Negative Rate:  0.8834355828220859
True Positive Rate:  0.6091549295774648
Balanced Accurancy:  0.7462952561997753
F1 Score:  0.8107513264972266


In [295]:
scaler = StandardScaler()
tree= DecisionTreeClassifier(random_state = 100472097)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

pipe = Pipeline(steps=[('scaler', scaler), ('tree', tree)])

param_grid = {
    'tree__max_depth': [3, 5, 7, None],
    'tree__min_samples_split': [2, 5, 10],
    'tree__min_samples_leaf': [1, 2, 4]
}

tree = GridSearchCV(pipe, param_grid, cv=TimeSeriesSplit(n_splits=5))
tree.fit(x_train_scaled, y_train)
y_pred = tree.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')

print("Mejores hiperparámetros encontrados:", tree.best_params_)
print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accuracy: ", balanced_accuracy)
print("F1 Score: ", f1)


Mejores hiperparámetros encontrados: {'tree__max_depth': 3, 'tree__min_samples_leaf': 2, 'tree__min_samples_split': 2}
Accuracy:  0.8462238398544131
Recall:  0.6795774647887324
True Negative Rate:  0.9042944785276074
True Positive Rate:  0.6795774647887324
Balanced Accuracy:  0.7919359716581699
F1 Score:  0.8450312236178545


# ***Matriz de confusión en AC***

La matriz de confusión es una herramienta útil para comprender el rendimiento del modelo en la clasificación de las clases.

In [296]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index = ["Real 0", "Real 1"], columns = [" Class Pred 0", "Class Pred 1"])

print(cm_df)

         Class Pred 0  Class Pred 1
Real 0            737            78
Real 1             91           193


En esta matriz de confusión:

Para la clase verdadera "0" (Real 0), el modelo predijo correctamente 737 instancias como "0" y erróneamente clasificó 78 instancias como "1".
Para la clase verdadera "1" (Real 1), el modelo predijo correctamente 193 instancias como "1" y erróneamente clasificó 91 instancias como "0".

# **MODELO REGRESION LOGISTICA**

Observaciones tras la obtención de resultados:

Los resultados de la regresión logística muestran que, incluso después de ajustar los hiperparámetros, no hubo una mejora significativa en las métricas de rendimiento. Aquí está el análisis comparativo:

**Sin ajustes de hiperparámetros:**
- Accuracy: 0.817
- Recall: 0.444
- True Negative Rate: 0.947
- True Positive Rate: 0.444
- Balanced Accuracy: 0.695
- F1 Score: 0.800

**Con ajustes de hiperparámetros:**
- Mejores hiperparámetros encontrados: {'logistic_regression__C': 10, 'logistic_regression__solver': 'liblinear'}
- Accuracy: 0.815
- Recall: 0.444
- True Negative Rate: 0.945
- True Positive Rate: 0.444
- Balanced Accuracy: 0.694
- F1 Score: 0.798

En ambos casos, los valores de las métricas de rendimiento (Accuracy, Recall, True Negative Rate, True Positive Rate, Balanced Accuracy y F1 Score) son muy similares. Esto sugiere que el ajuste de hiperparámetros no tuvo un impacto significativo en el rendimiento del modelo de regresión logística en este conjunto de datos.

En resumen, los ajustes de hiperparámetros no mejoraron el rendimiento del modelo de regresión logística en este caso específico. si recordamos algo similar pasaba en Regresión Lineal.

In [297]:
scaler = StandardScaler()
logistic_regression = LogisticRegression(random_state=100472097)

x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

logistic_regression.fit(x_train_scaled, y_train)
y_pred = logistic_regression.predict(x_test_scaled)

accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1_score = f1_score(y_test, y_pred, average='weighted')

print("Accuracy: ", accuracy)
print("Recall: ", recall)
print("True Negative Rate: ", tnr)  
print("True Positive Rate: ", tpr)
print("Balanced Accuracy: ", balanced_accuracy)
print("F1 Score: ", f1_score)


Accuracy:  0.8171064604185623
Recall:  0.44366197183098594
True Negative Rate:  0.947239263803681
True Positive Rate:  0.44366197183098594
Balanced Accuracy:  0.6954506178173334
F1 Score:  0.7999180747369808


In [298]:
from sklearn.metrics import f1_score

scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

# Inicializar el modelo de regresión logística
logistic_regression = LogisticRegression(random_state=100472097)

# Definir el pipeline con el escalador y el modelo
pipe = Pipeline(steps=[('scaler', scaler), ('logistic_regression', logistic_regression)])

# Definir los parámetros para la búsqueda de hiperparámetros
param_grid = {
    'logistic_regression__C': [0.1, 1, 10],
    'logistic_regression__solver': ['liblinear', 'newton-cg', 'lbfgs', 'sag', 'saga'],
}

# Inicializar GridSearchCV
logistic_regression_grid_search = GridSearchCV(pipe, param_grid, cv=5, scoring='balanced_accuracy', n_jobs=-1)

# Ajustar el modelo utilizando los datos de entrenamiento
logistic_regression_grid_search.fit(x_train_scaled, y_train)

# Realizar predicciones sobre el conjunto de prueba
y_pred = logistic_regression_grid_search.predict(x_test_scaled)

# Calcular métricas de rendimiento
accuracy = accuracy_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
tnr = recall_score(y_test, y_pred, pos_label=0)
tpr = recall_score(y_test, y_pred, pos_label=1)
balanced_accuracy = balanced_accuracy_score(y_test, y_pred)
f1_score_value = f1_score(y_test, y_pred, average='weighted')

# Imprimir resultados
print("Mejores hiperparámetros encontrados:", logistic_regression_grid_search.best_params_)
print("Accuracy:", accuracy)
print("Recall:", recall)
print("True Negative Rate:", tnr)
print("True Positive Rate:", tpr)
print("Balanced Accuracy:", balanced_accuracy)
print("F1 Score:", f1_score_value)


Mejores hiperparámetros encontrados: {'logistic_regression__C': 10, 'logistic_regression__solver': 'liblinear'}
Accuracy: 0.8152866242038217
Recall: 0.44366197183098594
True Negative Rate: 0.9447852760736196
True Positive Rate: 0.44366197183098594
Balanced Accuracy: 0.6942236239523027
F1 Score: 0.7983372420176964


# ***Matriz de confusión en RL***

La matriz de confusión es una herramienta útil para comprender el rendimiento del modelo en la clasificación de las clases.

In [299]:
cm = confusion_matrix(y_test, y_pred)
cm_df = pd.DataFrame(cm, index = ["Real 0", "Real 1"], columns = [" Class Pred 0", "Class Pred 1"])

print(cm_df)

         Class Pred 0  Class Pred 1
Real 0            770            45
Real 1            158           126


En esta matriz de confusión:

Para la clase verdadera "0" (Real 0), el modelo predijo correctamente 770 instancias como "0" y erróneamente clasificó 45 instancias como "1".
Para la clase verdadera "1" (Real 1), el modelo predijo correctamente 158 instancias como "1" y erróneamente clasificó 129 instancias como "0".

# **MEJOR MODELO DE CLASIFICACIÓN**

Aunque ya contasemos con los resultados obtenidos por el propio código decidimos tambien hacer los calculos a mano a partir de lo obtenido en las matrices de confusión. Entonces:

Para determinar cuál modelo es el mejor, podemos evaluar diferentes métricas de rendimiento, como la precisión, la sensibilidad (recall), la especificidad (True Negative/Possitive Rate), la precisión equilibrada (balanced accuracy) y la puntuación F1.

1. **Matriz de Confusión REGRESIÓN LOGISTICA:**
   - Precisión: (770 + 158) / (770 + 45 + 129 + 158) = 0.862
   - Sensibilidad para la clase 0: 770 / (770 + 45) = 0.945
   - Sensibilidad para la clase 1: 158 / (158 + 129) = 0.550
   - True Negative Rate: 770 / (770 + 45) = 0.945
   - True Positive Rate: 158 / (158 + 129) = 0.550
   - Precisión equilibrada: (0.945 + 0.550) / 2 = 0.748
   - Puntuación F1: (2 * 0.945 * 0.550) / (0.945 + 0.550) = 0.689

2. **Matriz de Confusión ÁRBOLES DE CLASIFICACIÓN:**
   - Precisión: (737 + 193) / (737 + 78 + 91 + 193) = 0.840
   - Sensibilidad para la clase 0: 737 / (737 + 78) = 0.904
   - Sensibilidad para la clase 1: 193 / (193 + 91) = 0.679
   - True Negative Rate: 737 / (737 + 78) = 0.904
   - True Positive Rate: 193 / (193 + 91) = 0.679
   - Precisión equilibrada: (0.904 + 0.679) / 2 = 0.791
   - Puntuación F1: (2 * 0.904 * 0.679) / (0.904 + 0.679) = 0.776

3. **Matriz de Confusión KNN:**
   - Precisión: (778 + 170) / (778 + 37 + 114 + 170) = 0.864
   - Sensibilidad para la clase 0: 778 / (778 + 37) = 0.954
   - Sensibilidad para la clase 1: 170 / (170 + 114) = 0.598
   - True Negative Rate: 778 / (778 + 37) = 0.954
   - True Positive Rate: 170 / (170 + 114) = 0.598
   - Precisión equilibrada: (0.954 + 0.598) / 2 = 0.776
   - Puntuación F1: (2 * 0.954 * 0.598) / (0.954 + 0.598) = 0.737

4. **Matriz de Confusión SVM:**
   - Precisión: (700 + 231) / (700 + 115 + 53 + 231) = 0.820
   - Sensibilidad para la clase 0: 700 / (700 + 115) = 0.859
   - Sensibilidad para la clase 1: 231 / (231 + 53) = 0.813
   - True Negative Rate: 700 / (700 + 115) = 0.859
   - True Positive Rate: 231 / (231 + 53) = 0.813
   - Precisión equilibrada: (0.859 + 0.813) / 2 = 0.836
   - Puntuación F1: (2 * 0.859 * 0.813) / (0.859 + 0.813) = 0.835

Basado en estas métricas:

- La **Matriz de Confusión KNN** tiene la mayor precisión equilibrada y una puntuación F1 más alta, lo que indica un mejor rendimiento general del modelo.
- La **Matriz de Confusión SVM** también tiene un rendimiento sólido en términos de precisión equilibrada y puntuación F1, siendo una buena opción también.
- La **Matriz de Confusión REGRESIÓN LOGÍSTICA** tiene la menor sensibilidad para la clase 1 y una puntuación F1 más baja, lo que indica que no clasifica tan bien la clase 1 como los otros modelos.

Por lo tanto, el Mejor Modelo sería el correspondiente a **la Matriz de Confusión KNN**, aunque tiene un True Positive Rate ligeramente más bajo que **la Matriz de Confusión SVM**.

LLegamos a la misma conclusión a partir de los datos obtenidos con el código:

Entonces para determinar el mejor modelo, necesitamos considerar todas las métricas de rendimiento proporcionadas para cada modelo (KNN, árboles de clasificación y regresión logística) tanto con ajustes como sin ajustes de hiperparámetros. Sin embargo, para simplificar, podemos usar una métrica compuesta, como el F1 Score ponderado, que tiene en cuenta la precisión y el recall de cada clase.

**SVM:**
- Sin ajuste de hiperparámetros: F1 Score ponderado = 0.863
- Con ajuste de hiperparámetros: F1 Score ponderado = 0.852

**KNN:**
- Sin ajustes de hiperparámetros: F1 Score ponderado = 0.664
- Con ajustes de hiperparámetros: F1 Score ponderado = 0.854

**Árboles de clasificación:**
- Sin ajustes de hiperparámetros: F1 Score ponderado = 0.811
- Con ajustes de hiperparámetros: F1 Score ponderado = 0.845

**Regresión logística:**
- Sin ajustes de hiperparámetros: F1 Score ponderado = 0.800
- Con ajustes de hiperparámetros: F1 Score ponderado = 0.798

El modelo SVM sin ajuste de hiperparámetros supera ligeramente a los otros modelos en términos de F1 Score ponderado, pero el modelo KNN con hiperparámetros ajustados sigue siendo el mejor modelo con un F1 Score ponderado de 0.854. Sin embargo, el modelo SVM con ajuste de hiperparámetros también muestra un rendimiento competitivo con un F1 Score ponderado de 0.852. Dependiendo de otros factores como el tiempo de entrenamiento y la interpretación del modelo, cualquiera de estos modelos podría considerarse como una opción viable.

Sin embargo nosotras tras realizar los calculos también a mano decimos quedarnos con el modelo KNN con hiperparámetros ajustados parece ser el mejor modelo para este conjunto de datos. Aunque volvemos a repetir, queda muy proximo a la SVM.

# 7. IMPLICACIONES DE CHATGPT

Durante el desarrollo de la práctica, nos vimos en la necesidad de recurrir a ChatGPT debido a la falta de conocimientos sólidos sobre el desarrollo del código y los objetivos específicos que estábamos tratando de alcanzar. Esta herramienta desempeñó un papel importante en varias áreas clave:

1. **Contextualización y comprensión de comandos**: ChatGPT nos ayudó a situarnos en el contexto de la práctica al proporcionarnos explicaciones claras sobre diversos comandos utilizados para la inserción y eliminación de datos en el archivo. Esta comprensión fue esencial para llevar a cabo las tareas de manera efectiva y eficiente.

2. **Limitaciones en la ayuda teórica**: Aunque también buscamos orientación teórica, encontramos ciertas limitaciones en la capacidad de ChatGPT para proporcionar respuestas precisas y relevantes. En algunas ocasiones, las respuestas se desviaban del tema central o no abordaban completamente nuestras preguntas, lo que dificultaba el proceso de aprendizaje. Sin embargo, cuando sucedía esto podiamos recurrir a las diapositivas que nos ayudan un poco más.

3. **Interpretación de resultados**: En momentos en que los resultados eran ambiguos o similares entre sí, ChatGPT fue útil para sugerir otras métricas o enfoques que podríamos considerar para tomar decisiones informadas. Su capacidad para ofrecer diferentes perspectivas nos permitió evaluar exhaustivamente los resultados y seleccionar la mejor opción.

4. **Contribución al proceso de codificación**: Si bien la participación de ChatGPT en el desarrollo del código no fue predominante, aproximadamente un 20% del proceso estuvo influenciado por sus sugerencias y explicaciones. Aunque comenzamos con el código proporcionado en clase, a menudo encontramos dificultades para comprenderlo completamente y adaptarlo a nuestras necesidades específicas. En estas situaciones, la claridad y orientación proporcionadas por ChatGPT fueron invaluables para superar los desafíos técnicos.

5. **Gestión de errores**: Una parte significativa de nuestra resolución de problemas estuvo vinculada a la ayuda de ChatGPT. Estimamos que alrededor del 40% de los errores detectados fueron resueltos gracias a sus explicaciones y sugerencias. En casos donde los errores eran complejos o no podíamos identificar la causa raíz, la capacidad de ChatGPT para proporcionar una orientación detallada fue crucial para encontrar soluciones efectivas.

En resumen, la utilización de ChatGPT en el desarrollo de la práctica fue fundamental para superar las limitaciones de nuestro conocimiento y abordar los desafíos técnicos de manera efectiva. Su capacidad para proporcionar orientación contextual, interpretar resultados y ayudar en la resolución de problemas contribuyó significativamente a nuestro proceso de aprendizaje y desarrollo.